In [2]:
pip install mip

Note: you may need to restart the kernel to use updated packages.


In [5]:
def kmerfind(seq,k):
        kmerList = []
        for i in range(0,len(seq)-k+1):
            kmer=seq[i:i+k]
            kmerList.append(kmer)
        return kmerList

In [6]:
def kmersort(kmerList):
    x=kmerList.copy()
    l = len(x)
    for i in range(l-1): 
        for j in range(l-i-1): 
            if x[j]>x[j+1]: 
                  x[j],x[j+1]=x[j+1],x[j]
    return x

In [7]:
from itertools import product
from sys import stdout as out
from mip import Model, xsum, minimize, BINARY
k = 3
target = 'CCAGATCTCTCGATCTTCTTACA'
kmerList = kmerfind(target,k) #splits sequence into kmers
places = kmersort(kmerList)# sorts the kmers in lexicographic order
# names of places to visit
# distances in an upper triangular matrix
dists = [[83, 81, 113, 52, 42, 73, 44, 23, 91, 105, 90, 124, 57],
         [161, 160, 39, 89, 151, 110, 90, 99, 177, 143, 193, 100],
         [90, 125, 82, 13, 57, 71, 123, 38, 72, 59, 82],
         [123, 77, 81, 71, 91, 72, 64, 24, 62, 63],
         [51, 114, 72, 54, 69, 139, 105, 155, 62],
         [70, 25, 22, 52, 90, 56, 105, 16],
         [45, 61, 111, 36, 61, 57, 70],
         [23, 71, 67, 48, 85, 29],
         [74, 89, 69, 107, 36],
         [117, 65, 125, 43],
         [54, 22, 84],
         [60, 44],
         [97],
         []]

# number of nodes and list of vertices
n, V = len(dists), set(range(len(dists)))

# distances matrix
c = [[0 if i == j
      else dists[i][j-i-1] if j > i
      else dists[j][i-j-1]
      for j in V] for i in V]

model = Model()

# binary variables indicating if arc (i,j) is used on the route or not
x = [[model.add_var(var_type=BINARY) for j in V] for i in V]

# continuous variable to prevent subtours: each city will have a
# different sequential id in the planned route except the first one
y = [model.add_var() for i in V]

# objective function: minimize the distance
model.objective = minimize(xsum(c[i][j]*x[i][j] for i in V for j in V))

# constraint : leave each city only once
for i in V:
    model += xsum(x[i][j] for j in V - {i}) == 1

# constraint : enter each city only once
for i in V:
    model += xsum(x[j][i] for j in V - {i}) == 1

# subtour elimination
for (i, j) in product(V - {0}, V - {0}):
    if i != j:
        model += y[i] - (n+1)*x[i][j] >= y[j]-n

# optimizing
model.optimize()

# checking if a solution was found
if model.num_solutions:
    out.write('route with total distance %g found: %s'
              % (model.objective_value, places[0]))
    nc = 0
    while True:
        nc = [i for i in V if x[nc][i].x >= 0.99][0]
        out.write(' -> %s' % places[nc])
        if nc == 0:
            break
    out.write('\n')

route with total distance 547 found: ACA -> AGA -> CAG -> CCA -> TAC -> CTT -> GAT -> ATC -> GAT -> CTT -> ATC -> CGA -> CTC -> CTC -> ACA
